In [80]:
from smoother.data.zod_data import ZodTrackingResults
from smoother.io.config_utils import load_config

#config = '/home/s0001668/workspace/thesis-autoann/AutoAnnSmoother/configs/training_config.yaml'
data_path = '/datasets/zod/zodv2'
result_path = '/staging/agp/masterthesis/2023autoann/storage/tracking/SimpleTrack_zod_full_train/results/Vehicle/results_lidar.json'
#result_path = '/staging/agp/masterthesis/2023autoann/storage/detection/CenterPoint/predictions/cp-zod-mini-results-train/pts_bbox/results_zod.json'
#model_path = '/staging/agp/masterthesis/2023autoann/storage/smoothing/autoannsmoothing/pc-track/pc-track_zod_train_pctrack-w5-sw_model.pth'
config = '/staging/agp/masterthesis/2023autoann/storage/smoothing/autoannsmoothing/pc-track/config.yaml'
model_path = '/staging/agp/masterthesis/2023autoann/storage/smoothing/autoannsmoothing/pc-track/pc-track_zod_train_pctrack-w25-sw_model.pth'
conf = load_config(config)
tracking_results = ZodTrackingResults(result_path, conf, 'full', 'train', data_path)


Initializing ZodData class...


Loading train sequences:   0%|          | 0/1165 [00:00<?, ?it/s]

Loading val sequences:   0%|          | 0/294 [00:00<?, ?it/s]

Loading prediction and ground-truths ...


100%|██████████| 1165/1165 [00:55<00:00, 20.90it/s]


In [81]:
from smoother.models.pc_track_net import PCTrackNet, TrackNet,PCNet 
import torch

model = PCTrackNet(track_encoder='pointnet', 
                pc_encoder='pointnet', 
                decoder='pool', 
                pc_feat_dim=4, 
                track_feat_dim=9, 
                pc_out=256, 
                track_out=64, 
                dec_out=16)

trained_model = model
checkpoint = torch.load(model_path)
trained_model.load_state_dict(checkpoint)

<All keys matched successfully>

In [82]:
import importlib
from smoother.data.common import transformations as t
importlib.reload(t)
from smoother.data.common import transformations as t

from smoother.data.common import tracking_data as td
importlib.reload(td)
from smoother.data.common import tracking_data as td

#center-offset means/stds
means = [-1.0408938187128578, 15.676385645189473, -0.5681853294492015, 2.5877273679628194, 1.0347822035160879, 0.9536876676924065, -0.3136310277276827, -0.013401212689278484]
stdev = [8.827627848858757, 19.47008377654335, 0.6877734215244837, 3.1708958713662074, 1.0381670406742867, 1.0549146278354404, 0.5951111005230132, 0.24225834247964198]
normalize = t.Normalize(means, stdev)

transformations = [t.ToTensor(), normalize, t.CenterOffset()]
tracking_data = td.TrackingData(tracking_results, transformations)

track_object = tracking_data.get(1)

100%|██████████| 1165/1165 [01:41<00:00, 11.45it/s]


In [83]:
data_model = td.WindowTrackingData(tracking_results, -12, 12, transformations, tracking_data)
track_object = data_model.get(0)
track_object

{'sequence_id': '000231', 'tracking_id': 'Vehicle_181_0', 'starting_frame_index': 0, 'track_length': 180, 'foi_index': 89, 'assoc_metric': 'giou', 'assoc_thresh': 0.0, 'has_gt': True, 'gt_dist': tensor(0.8270, dtype=torch.float64)}

In [84]:
trained_model.eval()

PCTrackNet(
  (pc_encoder): PCEncoder(
    (tnet4): TNet(
      (mlp): Sequential(
        (0): Linear(in_features=4, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=64, bias=True)
        (3): ReLU()
        (4): Linear(in_features=64, out_features=256, bias=True)
        (5): ReLU()
      )
      (fc): Linear(in_features=256, out_features=16, bias=True)
    )
    (conv1): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv4): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
  )
  (track_encoder): TrackEncoder(
    (tnet9): TNet(
      (mlp): Sequential(
        (0): Linear(in_features=9, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=64, bias=True)
        (3): ReLU()
        (4): Linear(in_features=64, out_features=64, bias=True)
        (5): ReLU()
      )
      (fc

In [86]:
len(data_model)

7461

In [94]:
from tqdm import tqdm
import os
from smoother.data.common.utils import convert_to_sine_cosine, convert_to_quaternion

zod = ZodSequences(data_path, 'full')

smoothing_results = {}
smoothing_results['meta'] = {
    'use_camera': False,
    'use_lidar': True,
     'use_radar': False,
    'use_map': False,
    'use_external': False
    }
smoothing_results['results'] = {}

for i, (x1, x2, y) in enumerate(data_model):
    if i % 100 == 0:
        print(i, '/', len(data_model))

    tracks, points, gt_anns = x1, x2, y #x1.to(self.device), x2.to(self.device), y.to(self.device)
    track_object = data_model.get(i)

    refined_track = trained_model.forward(tracks.unsqueeze(0), points.unsqueeze(0)).squeeze()
    for transformation in reversed(transformations):
        if type(transformation) == t.CenterOffset:
            transformation.set_offset(track_object.center_offset)
            transformation.set_start_and_end_index(0, -1)
        if type(transformation) == t.Normalize:
            transformation.set_start_and_end_index(0, -1)
        refined_track = transformation.untransform(refined_track)

    seq_id = track_object.sequence_id
    
    seq = zod[seq_id]
    frame_token = os.path.basename(seq.info.get_key_lidar_frame().filepath)

    refined_box = {
        'sample_token': frame_token,
        'translation': refined_track[:3].tolist(),
        'size': refined_track[3:6].tolist(),
        'rotation': convert_to_quaternion(refined_track[6:8].tolist()),
        'velocity': [0,0],
        'tracking_id': track_object.tracking_id,
        'tracking_name': 'Vehicle',
        'tracking_score': 0,
    }

    if seq_id not in smoothing_results['results']:
        smoothing_results['results'][frame_token] = []
    
    smoothing_results['results'][frame_token].append(refined_box)



/home/s0001668/workspace/thesis-autoann/AutoAnnSmoother/smoother/data/common/transformations.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.offset = torch.tensor(x[0:3], dtype=torch.float32)


0 / 7461
100 / 7461
200 / 7461
300 / 7461
400 / 7461
500 / 7461
600 / 7461
700 / 7461
800 / 7461
900 / 7461
1000 / 7461
1100 / 7461
1200 / 7461
1300 / 7461
1400 / 7461
1500 / 7461
1600 / 7461
1700 / 7461
1800 / 7461
1900 / 7461
2000 / 7461
2100 / 7461
2200 / 7461
2300 / 7461
2400 / 7461
2500 / 7461
2600 / 7461
2700 / 7461
2800 / 7461
2900 / 7461
3000 / 7461
3100 / 7461
3200 / 7461
3300 / 7461
3400 / 7461
3500 / 7461
3600 / 7461
3700 / 7461
3800 / 7461
3900 / 7461
4000 / 7461
4100 / 7461
4200 / 7461
4300 / 7461
4400 / 7461
4500 / 7461
4600 / 7461
4700 / 7461
4800 / 7461
4900 / 7461
5000 / 7461
5100 / 7461
5200 / 7461
5300 / 7461
5400 / 7461
5500 / 7461
5600 / 7461
5700 / 7461
5800 / 7461
5900 / 7461
6000 / 7461
6100 / 7461
6200 / 7461
6300 / 7461
6400 / 7461
6500 / 7461
6600 / 7461
6700 / 7461
6800 / 7461
6900 / 7461
7000 / 7461
7100 / 7461
7200 / 7461
7300 / 7461
7400 / 7461


In [97]:
len(smoothing_results['results'])

1088

In [95]:
import mmcv
save_path = '/staging/agp/masterthesis/2023autoann/storage/smoothing/autoannsmoothing/results/full_train.json'
mmcv.dump(smoothing_results, save_path)


In [71]:
print('foi', foi_box.center)
print('ref', refined_track[0:3])
print('gt', track_object.gt_box['translation'])

foi [4.994181518529014, 22.647279932471744, -1.550068295257373]
ref tensor([ 5.0743, 22.6973, -1.5385], grad_fn=<SliceBackward>)
gt [5.124, 22.317, -1.44]


In [ ]:

for track_id, refined_tracks in track_ids.items():
    


In [28]:
sample[0][13]

tensor([0.0000, 0.0000, 0.0000, 0.5593, 0.8350, 0.6963, 0.3102, 4.1486, 0.0000])

In [29]:
for boxes in tracking_results.pred_boxes['000000_quebec_2022-02-14T13:23:32.251875Z.npy']:
    for box in boxes:
        print(box['translation'])


[-33.513233184814474, 38.131393432617216, -6.40397644042969]
[-18.569011688232433, 38.04195404052737, -4.026726722717286]
[-36.35507965087893, 38.54131317138675, -5.940477371215822]
